Importing libraies and reading data csv file into a Pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
df_tor = pd.read_csv(body)
df_tor.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Rename columns

In [4]:
df_tor.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Remove all rows that have a Borough Not Assigned

In [5]:
df_tor['Borough'].replace('Not assigned',np.nan, inplace=True)
df_tor.dropna(subset=['Borough'], axis=0, inplace=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Combine rows with the same Postal Code <br/>
Replace Not Assigned Neighborood values with their corresponding Borough value <br/>
Randomize dataframe rows

In [6]:
df_tor=df_tor.groupby(['PostalCode','Borough'], as_index =False).agg({'Neighborhood': ', '.join})
df_tor['Neighborhood'].replace('Not assigned',df_tor['Borough'], inplace=True)
df_tor = df_tor.sample(frac=1).reset_index(drop=True)
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M4B,East York,"Woodbine Gardens, Parkview Hill"
1,M6H,West Toronto,"Dovercourt Village, Dufferin"
2,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ..."
3,M4J,East York,East Toronto
4,M6P,West Toronto,"High Park, The Junction South"


In [7]:
df_tor.shape

(103, 3)

The datframe has 103 rows or records and 3 columns or features

In [8]:
body = client_d6b108fb8f9a44b6b683b7059b7338a1.get_object(Bucket='myfirstproject-donotdelete-pr-qxcxkzyfq9az74',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_coords = pd.read_csv(body)
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Join the coordinates dataframe with the original dataframe on the matching Postal Code of both dataframes

In [9]:
df_tor=df_tor.join(df_coords.set_index('Postal Code'), on='PostalCode')
df_tor.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
1,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
2,M9B,Etobicoke,"Cloverdale, Islington, Martin Grove, Princess ...",43.650943,-79.554724
3,M4J,East York,East Toronto,43.685347,-79.338106
4,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1S,Scarborough,Agincourt,43.794200,-79.262029
7,M1W,Scarborough,L'Amoreaux West,43.799525,-79.318389
8,M4C,East York,Woodbine Heights,43.695344,-79.318389
9,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420


In [10]:
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Narrow down the datframe to only records whose boroughs contain the word Toronto

In [14]:
tor_data = df_tor[df_tor['Borough'].str.contains('Toronto')].reset_index(drop=True)

Defining a user_agent for an instance of the geocoder centered at the City of Toronto

In [28]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Created a map of Toronto using corridnates </br>
Created markers for each cluster of neighborhoods that have the same Postal Code
Labeled each cluster with list of neighbrhoods, borough, and Postal Code

In [30]:
map_tor = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, postalcode, borough, neighborhood in zip(tor_data['Latitude'], tor_data['Longitude'], tor_data['PostalCode'], tor_data['Borough'], tor_data['Neighborhood']):
    label = '{}, {} {}'.format(neighborhood, borough, postalcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

You can observe there are more clusters of neighborhoods the closer we get to the central coordinate of downtown Toronto